In [ ]:
from functions import get_current_location
location = get_current_location()
if location:
    latitude, longitude = location
    print(f'Current Latitude: {latitude}, Longitude: {longitude}')
else:
    print('Failed to determine current location.')

In [ ]:
from route_csv2df import RouteDF
routeDF = RouteDF()

from api_route import ApiRoute
current_position = {'longitude': 130.846642,
                    'latitude': -12.4534}
current_position = {'longitude': 137.80163,
                    'latitude': -32.51756}
final_position = {'longitude': 130.890346,
                  'latitude': -12.432393}
delta_spacing = 2000.0 # in meters
number_sites = 10

route = ApiRoute(routeDF.get_route_data, print_is_requested=True)

# route_df = route.cut_route_data(current_position, number_sites=number_sites)
route_df = route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing)
# route_df = route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites)



In [ ]:
from api_executer import ApiExecuter
from api_requester import ApiRequester
from api_parser import ApiParser

api_parser = ApiParser()
api_requester = ApiRequester(parser=api_parser, print_is_requested=True)
api_executer = ApiExecuter(requester=api_requester)

In [ ]:
api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [ ]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(route_df, print_is_requested=True)

In [ ]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [ ]:
import pandas as pd
# import numpy as np
# import random
# import string

sites_list = api_executer.get_all_site_id
s_len = len(sites_list)

# # Create the DataFrame
# prob = 1.0
# data = {
#     # 'name': ['e'+str(random.randint(1,10)) for _ in range(s_len)],
#     'latitude': [random.uniform(-90.0, 90.0) if random.random() < prob else np.nan for _ in range(s_len)],
#     'longitude': [random.uniform(-180.0, 180.0) if random.random() < prob else np.nan for _ in range(s_len)]
# }

data = {
    'latitude': [route_df['latitude'][line] for line in range(s_len)],
    'longitude': [route_df['longitude'][line] for line in range(s_len)]
}

edit_sites = pd.DataFrame(data, index=sites_list)
print(edit_sites)

api_executer.edit_sites(edit_sites)

In [ ]:
# raw_forecast_df = api_requester.get_solar_forecast()
raw_forecast_df = api_requester.get_solar_forecast_cloudmove()
print(raw_forecast_df)

In [ ]:
sites_df = api_requester.forecast_sites
print(sites_df)

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = 4
forecast_df, preprocessed_df = preprocessor.preprocess(route_df, sites_df, raw_forecast_df, hours_in_advance)
preprocessor.save_data2folder()
print(preprocessed_df)